In [5]:
import sys
sys.path.append('..')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from interface.load_data import (
    load_data_fault_free_test,
    load_data_fault_free_train,
    load_data_faulty_test,
    load_data_faulty_train,
    load_data_fault_free_test_select

)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [6]:
from keras import Sequential, Input, layers
from keras.callbacks import EarlyStopping

In [7]:
faulty_train = load_data_faulty_train()
faulty_train.shape

(5000000, 55)

In [8]:
mask = [i*10 for i in range(int(faulty_train.shape[0]/10))]

faulty_train_w = faulty_train.iloc[mask]
faulty_train_w.shape

(500000, 55)

In [9]:
X = faulty_train.drop(columns=['faultNumber', 'simulationRun', 'sample'])
y = faulty_train.faultNumber

In [10]:
X_w = faulty_train_w.drop(columns=['faultNumber', 'simulationRun', 'sample'])
y_w = faulty_train_w.faultNumber

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_w, y_w, test_size=0.5)

In [31]:
scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

In [32]:
n_simulation = 500
len_simulation = 10000
n_per_sample = 50
n_fault = 20
n_features = 52

In [33]:
X_train_reshaped = X_train_scaled.values.reshape(round(len_simulation*0.5), n_per_sample, n_features)
y_train_reshaped = y_train.values.reshape(round(len_simulation*0.5), n_per_sample)[:, 0]

X_test_reshaped = X_test_scaled.values.reshape(round(len_simulation*0.5), n_per_sample, n_features)
y_test_reshaped = y_test.values.reshape(round(len_simulation*0.5), n_per_sample)[:, 0]

In [34]:
print(X_train_reshaped.shape)
print(y_train_reshaped.shape)

(5000, 50, 52)
(5000,)


In [41]:
model = Sequential([
    Input(shape=(50, 52)),

    layers.Conv1D(64, kernel_size=7, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(128, kernel_size=5, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(256, kernel_size=3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(512, kernel_size=3, activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),

    layers.Flatten(),

    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),

    layers.Dense(21, activation='softmax')
])

In [42]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [43]:
callbacks = [EarlyStopping(patience=15)]

history = model.fit(
    X_train_reshaped, y_train_reshaped,
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
)

Epoch 1/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.0570 - loss: 3.2779
Epoch 2/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.0748 - loss: 3.0260
Epoch 3/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.1070 - loss: 2.8860
Epoch 4/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.1260 - loss: 2.8447
Epoch 5/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.1320 - loss: 2.8119
Epoch 6/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.1528 - loss: 2.7028
Epoch 7/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.1798 - loss: 2.6352
Epoch 8/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2034 - loss: 2.5675
Epoch 9/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.2180 - loss: 2.5035
Epoch 10/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2508 - loss: 2.4144
Epoch 11/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.2664 - loss: 2.3513
Epoch 12/100
157/157 ━━━━━━━━━

In [40]:
model.evaluate(X_test_reshaped, y_test_reshaped)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1940 - loss: 8.3305


[8.330547332763672, 0.1940000057220459]